In [125]:
key_path = '../secrets/sheet-reader-key.json'
spreadsheet_id = '1VwC-GYUuEjsZr1DqGgeo8Wkv6_Ly6NZjAV07Zeik4ko'


In [312]:
key_path = '../secrets/sheet-reader-key.json'
spreadsheet_id = '1VwC-GYUuEjsZr1DqGgeo8Wkv6_Ly6NZjAV07Zeik4ko'


import os

from apiclient import discovery
from google.oauth2 import service_account

scopes = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file', 'https://www.googleapis.com/auth/spreadsheets']
secret_file = os.path.join(os.getcwd(), key_path)


credentials = service_account.Credentials.from_service_account_file(secret_file, scopes=scopes)
service = discovery.build('sheets', 'v4', credentials=credentials)

In [463]:
def schedule_sheets_update(mode):
    if mode not in ['create','delete']:
        raise ValueError('Invalid mode')
    range_name = 'AIKATAULU!A1:D100'
    template_id = '61864045'
    
    schedule = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name, valueRenderOption='UNFORMATTED_VALUE').execute()
    
    metadata = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    all_sheets = {p['properties']['title']:p['properties']['sheetId'] for p in metadata['sheets']}
    
    n_games = {}
    reqs = []
    
    update_cells = []
    for i, row in enumerate(schedule['values'][1:]):
        if len(row) == 4:
            teams = sorted(row[1:3])
            game_id = f'{teams[0]}{teams[1]}{row[0]}'
            n_games[game_id] = n_games.get(game_id, 0) + 1

            name = f'{game_id}_{n_games[game_id]}'
            update_cells.append((name, True))

            if (mode == 'create'):
                if name not in all_sheets:
                    reqs.append({
                        "duplicateSheet": {
                            "sourceSheetId": template_id,
                            "newSheetName": name,
                        }
                    })                
                
            elif (mode=='delete' and name in all_sheets):
                reqs.append({
                    f"deleteSheet": {
                        "sheetId": all_sheets[name]
                    }
                })
        else:
            update_cells.append(('', False))

    if len(reqs) > 0:# Execute the batchUpdate request
        body = {"requests": reqs}
        response = service.spreadsheets().batchUpdate(
            spreadsheetId=spreadsheet_id,
            body=body
        ).execute()
        
        metadata = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
        all_sheets = {p['properties']['title']:p['properties']['sheetId'] for p in metadata['sheets']}
    
   
    cell_updates = []
    if mode == 'create':
        url_base = f"""=HYPERLINK(\"https://docs.google.com/spreadsheets/d/{spreadsheet_id}/edit"""
        names = [r['duplicateSheet']['newSheetName'] for r in reqs]
        for name, update in update_cells:
            if update:
                cell_updates.append([f"""{url_base}?gid={all_sheets[name]}#gid={all_sheets[name]}", "Gamelink")"""])
            else:
                cell_updates.append([""])

    if mode == 'delete':
        cell_updates = [[""] for i in schedule['values']]
    
    body = {
        'values': cell_updates
    }

    response = service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range="AIKATAULU!E2:E",  # Specify the range
        valueInputOption="USER_ENTERED",
        body=body
    ).execute()
    
    if mode == 'create':
        data = []
        for i, row in enumerate(schedule['values'][1:]):
            
            if len(row) == 4:   
                teams = sorted(row[1:3])
                game_id = f'{teams[0]}{teams[1]}{row[0]}'
                for i in range(1, n_games[game_id]+1):
                    name = f'{game_id}_{i}'
                    data.append([
                        {
                            'range': f'{name}!A2:A2',  # Specify the first range
                            'values': [[row[1]]]
                        },
                        {
                            'range': f'{name}!F2:F2',  # Specify the first range
                            'values': [[row[2]]]
                        }
                    ])
        response = service.spreadsheets().values().batchUpdate(
            spreadsheetId=spreadsheet_id,
            body={'data':data, 'valueInputOption':'USER_ENTERED'}
        ).execute()
    
    
    move_requests = []
    for sheet_name in ['AIKATAULU','JOUKKUEET','NHL','TEMPLATE']:
        move_requests.append({
        "updateSheetProperties": {
            "properties": {
                "sheetId": all_sheets[sheet_name],
                "index": 0
            },
            "fields": "index"
        }})
    
    request_body_move = {
        "requests": [move_requests]
    }
    service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body=request_body_move
    ).execute()

    return
    

In [464]:
import pandas as pd
def read_data(range_name):
    data = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name, valueRenderOption='UNFORMATTED_VALUE').execute()
    return pd.DataFrame(data['values'][1:], columns=data['values'][0])

def read_players():
    range_name = 'NHL!A1:T'  
    return read_data(range_name)

def read_teams():
    range_name = 'JOUKKUEET!A1:B'  
    return read_data(range_name)

def read_schedule():
    range_name = 'AIKATAULU!A1:D'
    
    return read_data(range_name).dropna()

def read_players():
    range_name = 'NHL!A1:T'  
    return read_data(range_name)




In [465]:
def read_game_data():
    n_games = {}
    metadata_rows = []
    ranges = []
    for row in schedule.to_dict(orient='records'):
        teams = sorted([row['KOTI'], row['VIERAS']])
        game_id = f"{teams[0]}{teams[1]}{row['SARJA']}"
        n_games[game_id] = n_games.get(game_id, 0) + 1

        name = f'{game_id}_{n_games[game_id]}'
        row['name'] = name
        ranges.append(f'{name}!A5:J30')
        metadata_rows.append(row)


    # Request to batch get values from multiple ranges
    response = service.spreadsheets().values().batchGet(
        spreadsheetId=spreadsheet_id,
        ranges=ranges,
        valueRenderOption='UNFORMATTED_VALUE'
    ).execute()

    goals = []
    for value_range, metadata in zip(response['valueRanges'], metadata_rows):
        metadata['game_state'] = value_range['values'][23][3]
        for i, row in enumerate(value_range.get('values', [])[:20]):
            if len(row) > 0:
                row_data = {
                    'scoring_team':'KOTI',
                    'scorer':row[0],
                    'ass_1': row[1],
                    'ass_2': row[2],
                    'n_goals': i + 1
                }

                row_data.update(metadata)
                goals.append(row_data)

                row_data = {
                    'scoring_team':'VIERAS',
                    'scorer':row[5],
                    'ass_1': row[6],
                    'ass_2': row[7],
                    'n_goals': i + 1
                }
                row_data.update(metadata)
                goals.append(row_data)
    goals = pd.DataFrame(goals)
    goals = goals[goals.scorer != '']
    return pd.DataFrame(metadata_rows), goals



In [466]:
# schedule_sheets_update('delete')
# schedule_sheets_update('create')
# players = read_players()
# players
# teams = read_teams()
schedule = read_schedule()


In [493]:

def get_scoreboard(games, goals):
    scores = goals.groupby(['scoring_team', 'SARJA', 'KOTI', 'VIERAS', 'name'], as_index=False)['n_goals'].max()
    scores = scores.pivot(columns='scoring_team', index='name', values='n_goals')

    if len(goals) > 0:
        scoreboard = games.join(scores, on=['name'], rsuffix='_SCORE')
    else:
        scoreboard = games
        scoreboard['KOTI_SCORE'] = 0
        scoreboard['VIERAS_SCORE'] = 0

    scoreboard[['KOTI_SCORE', 'VIERAS_SCORE']] = scoreboard[['KOTI_SCORE', 'VIERAS_SCORE']].fillna(0)
    scoreboard['game_n'] = ''
    scoreboard.loc[scoreboard['SARJA'] == 'Playoffs', 'game_n'] = 'Playoffs (Game ' + scoreboard['name'].str.split('_').str[1] + '.)'
    scoreboard['series'] = scoreboard['name'].str[:-2]
    scoreboard.loc[scoreboard.series.str.endswith('Runkosarja'), 'series'] = 'Runkosarja'
    scoreboard['winner'] = scoreboard.apply(
        lambda row: row['KOTI'] if row['KOTI_SCORE'] > row['VIERAS_SCORE'] else 
                    (row['VIERAS'] if row['VIERAS_SCORE'] > row['KOTI_SCORE'] else 'TIE'), 
        axis=1
    )
    return scoreboard

def get_standings(scoreboard):    # Transform to long-form and count victories as before
    long_df = pd.melt(
        scoreboard.loc[scoreboard.game_state == 'PÄÄTTYNYT'], 
        id_vars=["series", "winner", "KOTI_SCORE", "VIERAS_SCORE"], 
        value_vars=["KOTI", "VIERAS"],
        value_name="team"
    )
    
    long_df['win'] = long_df['team'] == long_df['winner']
    long_df['tie'] = long_df['winner'] == 'TIE'
    long_df['loss'] = (~long_df['win'])&(~long_df['tie'])
    long_df['goalsfor'] = 0
    long_df.loc[long_df.variable == 'KOTI', 'goalsfor'] += long_df.loc[long_df.variable == 'KOTI', 'KOTI_SCORE']
    long_df.loc[long_df.variable == 'VIERAS', 'goalsfor'] += long_df.loc[long_df.variable == 'VIERAS', 'VIERAS_SCORE']
    long_df['goalsaga'] = 0
    long_df.loc[long_df.variable == 'KOTI', 'goalsaga'] += long_df.loc[long_df.variable == 'KOTI', 'VIERAS_SCORE']
    long_df.loc[long_df.variable == 'VIERAS', 'goalsaga'] += long_df.loc[long_df.variable == 'VIERAS', 'KOTI_SCORE']
    standings = long_df.groupby(['series','team'], as_index=False)[['win','tie','loss']].sum()
    standings['points'] = standings['win'] * 2 + standings['tie']
    return standings

In [494]:
games, goals = read_game_data()
get_scoreboard(games, goals)

,SARJA,KOTI,VIERAS,AREENA,name,game_state,KOTI_SCORE,VIERAS_SCORE,game_n,series,winner
0,Runkosarja,Antti,Arttu,XBOX,AnttiArttuRunkosarja_1,TULOSSA,0.0,0.0,,Runkosarja,TIE
1,Runkosarja,Juuso,Lauru,PS3,JuusoLauruRunkosarja_1,TULOSSA,0.0,0.0,,Runkosarja,TIE
2,Runkosarja,Tuomo,Arttu,XBOX,ArttuTuomoRunkosarja_1,TULOSSA,0.0,0.0,,Runkosarja,TIE
3,Runkosarja,Vesa,Joel,PS3,JoelVesaRunkosarja_1,TULOSSA,0.0,0.0,,Runkosarja,TIE
4,Runkosarja,Riku,Arvo,XBOX,ArvoRikuRunkosarja_1,TULOSSA,0.0,0.0,,Runkosarja,TIE
5,Runkosarja,Vessu,Vesa,PS3,VesaVessuRunkosarja_1,KÄYNNISSÄ,1.0,1.0,,Runkosarja,TIE
6,Runkosarja,Joel,Lauru,XBOX,JoelLauruRunkosarja_1,PÄÄTTYNYT,0.0,0.0,,Runkosarja,TIE
7,Runkosarja,Arvo,Joel,PS3,ArvoJoelRunkosarja_1,PÄÄTTYNYT,1.0,1.0,,Runkosarja,TIE
8,Playoffs,Arttu,Lauru,PS3,ArttuLauruPlayoffs_1,PÄÄTTYNYT,1.0,2.0,Playoffs (Game 1.),ArttuLauruPlayoffs,Lauru
9,Playoffs,Lauru,Arttu,XBOX,ArttuLauruPlayoffs_2,KÄYNNISSÄ,3.0,4.0,Playoffs (Game 2.),ArttuLauruPlayoffs,Arttu


In [495]:
standings = get_standings(scoreboard)
standings

,series,team,win,tie,loss,points
0,ArttuLauruPlayoffs,Arttu,0,0,1,0
1,ArttuLauruPlayoffs,Lauru,1,0,0,2
2,Runkosarja,Arvo,0,1,0,1
3,Runkosarja,Joel,1,0,0,2
4,Runkosarja,Lauru,0,0,1,0
5,Runkosarja,Riku,0,1,0,1
6,Runkosarja,Vesa,0,1,0,1
7,Runkosarja,Vessu,0,1,0,1


In [451]:
long_df

,series,winner,variable,team,win,tie,loss
0,Runkosarja,TIE,KOTI,Riku,False,True,False
1,Runkosarja,TIE,KOTI,Vessu,False,True,False
2,Runkosarja,Joel,KOTI,Joel,True,False,False
3,ArttuLauruPlayoffs,Lauru,KOTI,Arttu,False,False,True
4,Runkosarja,TIE,VIERAS,Arvo,False,True,False
5,Runkosarja,TIE,VIERAS,Vesa,False,True,False
6,Runkosarja,Joel,VIERAS,Lauru,False,False,True
7,ArttuLauruPlayoffs,Lauru,VIERAS,Lauru,True,False,False


In [449]:
standings[standings.series == 'Runkosarja'].sort_values('points', ascending=False)

,series,team,win,tie,loss,points
3,Runkosarja,Joel,1,0,0,2
2,Runkosarja,Arvo,0,1,0,1
5,Runkosarja,Riku,0,1,0,1
6,Runkosarja,Vesa,0,1,0,1
7,Runkosarja,Vessu,0,1,0,1
4,Runkosarja,Lauru,0,0,1,0


,RUNKOSARJA/PLAYOFFS,KOTI,VIERAS,AREENA,name,game_state
0,Runkosarja,Antti,Arttu,XBOX,AnttiArttuRunkosarja_1,TULOSSA
1,Runkosarja,Juuso,Lauru,PS3,JuusoLauruRunkosarja_1,TULOSSA
2,Runkosarja,Tuomo,Arttu,XBOX,TuomoArttuRunkosarja_1,TULOSSA
3,Runkosarja,Vesa,Joel,PS3,VesaJoelRunkosarja_1,TULOSSA
4,Runkosarja,Riku,Arvo,XBOX,RikuArvoRunkosarja_1,TULOSSA


,RUNKOSARJA/PLAYOFFS,KOTI,VIERAS,AREENA,name,game_state,n_goals
0,Runkosarja,Antti,Arttu,XBOX,AnttiArttuRunkosarja_1,TULOSSA,NaN
1,Runkosarja,Juuso,Lauru,PS3,JuusoLauruRunkosarja_1,TULOSSA,NaN
2,Runkosarja,Tuomo,Arttu,XBOX,TuomoArttuRunkosarja_1,TULOSSA,NaN
3,Runkosarja,Vesa,Joel,PS3,VesaJoelRunkosarja_1,TULOSSA,NaN
4,Runkosarja,Riku,Arvo,XBOX,RikuArvoRunkosarja_1,TULOSSA,NaN
5,Runkosarja,Vessu,Vesa,PS3,VessuVesaRunkosarja_1,TULOSSA,NaN
6,Runkosarja,Joel,Lauru,XBOX,JoelLauruRunkosarja_1,TULOSSA,NaN
7,Runkosarja,Arvo,Joel,PS3,ArvoJoelRunkosarja_1,TULOSSA,NaN
8,Playoffs,Arttu,Lauru,PS3,ArttuLauruPlayoffs_1,TULOSSA,NaN
9,Playoffs,Lauru,Arttu,XBOX,LauruArttuPlayoffs_1,TULOSSA,NaN


In [293]:
game_scores.loc[game_scores.game_state == 'KÄYNNISSÄ']

,RUNKOSARJA/PLAYOFFS,KOTI,VIERAS,AREENA,name,game_state,KOTI_,VIERAS_


In [287]:
games

,RUNKOSARJA/PLAYOFFS,KOTI,VIERAS,AREENA,name,game_state
0,Runkosarja,Antti,Arttu,XBOX,AnttiArttuRunkosarja_1,TULOSSA
1,Runkosarja,Juuso,Lauru,PS3,JuusoLauruRunkosarja_1,TULOSSA
2,Runkosarja,Tuomo,Arttu,XBOX,TuomoArttuRunkosarja_1,TULOSSA
3,Runkosarja,Vesa,Joel,PS3,VesaJoelRunkosarja_1,TULOSSA
4,Runkosarja,Riku,Arvo,XBOX,RikuArvoRunkosarja_1,TULOSSA
5,Runkosarja,Vessu,Vesa,PS3,VessuVesaRunkosarja_1,TULOSSA
6,Runkosarja,Joel,Lauru,XBOX,JoelLauruRunkosarja_1,TULOSSA
7,Runkosarja,Arvo,Joel,PS3,ArvoJoelRunkosarja_1,TULOSSA
8,Playoffs,Arttu,Lauru,PS3,ArttuLauruPlayoffs_1,TULOSSA
9,Playoffs,Lauru,Arttu,XBOX,LauruArttuPlayoffs_1,TULOSSA


ValueError: columns overlap but no suffix specified: Index(['KOTI', 'VIERAS'], dtype='object')